In [1]:
using Turing
using LimberJack
using CSV
using NPZ
using Plots
using FITSIO
using Plots
using LinearAlgebra
using Distributed
using StatsPlots
using BenchmarkTools

In [2]:
Threads.nthreads()

4

In [3]:
files = npzread("../data/DESY1_cls/Cls_meta.npz")
Cls_meta = cls_meta(files);

In [4]:
cov_tot = files["cov"]
inv_cov_tot = files["inv_cov"]
data_vector = files["cls"];

In [5]:
nuisances = Dict("b0" => 1.41,
                 "b1" => 1.62,
                 "b2" => 1.60,
                 "b3" => 1.92,
                 "b4" => 2.00)

cosmology = LimberJack.Cosmology(0.25, 0.05, 0.67, 0.96, 0.81,
                                 tk_mode="EisHu",
                                 Pk_mode="Halofit");

In [6]:
@benchmark Theory(cosmology, nuisances, Cls_meta, files)

BenchmarkTools.Trial: 
  memory estimate:  706.25 MiB
  allocs estimate:  43274549
  --------------
  minimum time:     1.696 s (5.73% GC)
  median time:      1.852 s (5.64% GC)
  mean time:        1.814 s (5.65% GC)
  maximum time:     1.894 s (5.51% GC)
  --------------
  samples:          3
  evals/sample:     1

In [7]:
@time Theory(cosmology, nuisances, Cls_meta, files)

  1.877236 seconds (43.27 M allocations: 706.247 MiB, 6.61% gc time)


35-element Vector{Any}:
 [0.0002676186708586752, 8.841546375620103e-5, 4.047842996445243e-5, 2.517648848261276e-5, 1.719685318428818e-5]
 [0.00014154427725827532, 6.082965496138611e-5, 3.790912786833451e-5, 2.1105066816464778e-5, 1.50417822905421e-5, 1.0832740348411216e-5, 8.412670337432821e-6, 6.842558212033605e-6]
 [7.323117603985917e-5, 4.5440711342447205e-5, 2.6346180179657362e-5, 1.8925694986657402e-5, 1.1868694138595653e-5, 9.106928364464173e-6, 7.024827283867212e-6, 5.360095302509174e-6, 4.518132908657019e-6, 3.767214915663323e-6]
 [5.577733457946745e-5, 4.616282106211585e-5, 2.5094737902377355e-5, 1.8921595296841637e-5, 1.4679950450745867e-5, 9.80384794911678e-6, 7.528717771972302e-6, 6.39721752666294e-6, 5.058527350410401e-6, 4.07390666682515e-6, 3.4853684717331534e-6]
 [3.950129104496599e-5, 3.812504259743051e-5, 2.4652125155126748e-5, 1.6314431346226586e-5, 1.4146989537770204e-5, 1.1438017014359918e-5, 7.108417200822868e-6, 5.733138823700403e-6, 4.96492787875246e-6, 4.266577

In [8]:
@benchmark Theory_parallel(cosmology, nuisances, Cls_meta, files)

BenchmarkTools.Trial: 
  memory estimate:  706.24 MiB
  allocs estimate:  43274926
  --------------
  minimum time:     1.432 s (13.68% GC)
  median time:      1.674 s (13.29% GC)
  mean time:        1.738 s (13.47% GC)
  maximum time:     2.172 s (13.94% GC)
  --------------
  samples:          4
  evals/sample:     1

In [9]:
@model function model_parallel(data_vector; cov_tot=cov_tot)
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 ~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)
    
    b0 = 1.69 #1.41 #~ Uniform(1.0, 3.0)
    b1 = 2.05 #1.62 #~ Uniform(1.0, 3.0)
    b2 = 2.01 #1.60 #~ Uniform(1.0, 3.0)
    b3 = 2.46 #1.92 #~ Uniform(1.0, 3.0)
    b4 = 2.54 #2.00 #~ Uniform(1.0, 3.0)
    
    #dz_g0 ~ Normal(0.008, 0.007)
    #dz_g1 ~ Normal(-0.005, 0.007)
    #dz_g2 ~ Normal(0.006, 0.006)
    #dz_g3 ~ Normal(0.0, 0.010)
    #dz_g4 ~ Normal(0.0, 0.010)
    
    #dz_k0 ~ Normal(-0.001, 0.016)
    #dz_k1 ~ Normal(-0.019, 0.013)
    #dz_k2 ~ Normal(-0.009, 0.011)
    #dz_k3 ~ Normal(-0.018, 0.022)
    
    #m0 ~ Normal(0.0, 0.035)
    #m1 ~ Normal(0.0, 0.035)
    #m2 ~ Normal(0.0, 0.035)
    #m3 ~ Normal(0.0, 0.035)
    
    #A_IA ~ Uniform(-5, 5) 
    #alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("b0" => b0,
                     "b1" => b1,
                     "b2" => b2,
                     "b3" => b3,
                     "b4" => b4) #,
                     #"dz_g0" => dz_g0,
                     #"dz_g1" => dz_g1,
                     #"dz_g2" => dz_g2,
                     #"dz_g3" => dz_g3,
                     #"dz_g4" => dz_g4,
                     #"dz_k0" => dz_k0,
                     #"dz_k1" => dz_k1,
                     #"dz_k2" => dz_k2,
                     #"dz_k3" => dz_k3,
                     #"m0" => m0,
                     #"m1" => m1,
                     #"m2" => m2,
                     #"m3" => m3,
                     #"A_IA" => A_IA,
                     #"alpha_IA" => alpha_IA)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = vcat(Theory_parallel(cosmology, nuisances, Cls_meta, files)...)
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [10]:
@model function model_linear(data_vector; cov_tot=cov_tot)
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 ~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)
    
    b0 = 1.69 #1.41 #~ Uniform(1.0, 3.0)
    b1 = 2.05 #1.62 #~ Uniform(1.0, 3.0)
    b2 = 2.01 #1.60 #~ Uniform(1.0, 3.0)
    b3 = 2.46 #1.92 #~ Uniform(1.0, 3.0)
    b4 = 2.54 #2.00 #~ Uniform(1.0, 3.0)
    
    #dz_g0 ~ Normal(0.008, 0.007)
    #dz_g1 ~ Normal(-0.005, 0.007)
    #dz_g2 ~ Normal(0.006, 0.006)
    #dz_g3 ~ Normal(0.0, 0.010)
    #dz_g4 ~ Normal(0.0, 0.010)
    
    #dz_k0 ~ Normal(-0.001, 0.016)
    #dz_k1 ~ Normal(-0.019, 0.013)
    #dz_k2 ~ Normal(-0.009, 0.011)
    #dz_k3 ~ Normal(-0.018, 0.022)
    
    #m0 ~ Normal(0.0, 0.035)
    #m1 ~ Normal(0.0, 0.035)
    #m2 ~ Normal(0.0, 0.035)
    #m3 ~ Normal(0.0, 0.035)
    
    #A_IA ~ Uniform(-5, 5) 
    #alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("b0" => b0,
                     "b1" => b1,
                     "b2" => b2,
                     "b3" => b3,
                     "b4" => b4) #,
                     #"dz_g0" => dz_g0,
                     #"dz_g1" => dz_g1,
                     #"dz_g2" => dz_g2,
                     #"dz_g3" => dz_g3,
                     #"dz_g4" => dz_g4,
                     #"dz_k0" => dz_k0,
                     #"dz_k1" => dz_k1,
                     #"dz_k2" => dz_k2,
                     #"dz_k3" => dz_k3,
                     #"m0" => m0,
                     #"m1" => m1,
                     #"m2" => m2,
                     #"m3" => m3,
                     #"A_IA" => A_IA,
                     #"alpha_IA" => alpha_IA)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = vcat(Theory_parallel(cosmology, nuisances, Cls_meta, files)...)
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [11]:
iterations = 100
nadapts = 20
TAP = 0.60
nchains = 4

4

In [11]:
chains_linear = sample(model_linear(data_vector), NUTS(TAP, nadapts),
                       iterations, progress=true, save_state=true)

LoadError: BoundsError: attempt to access Float64 at index [2]

In [12]:
chains_parallel = sample(model_parallel(data_vector), NUTS(TAP, nadapts),
                       iterations, progress=true, save_state=true)

LoadError: BoundsError: attempt to access Float64 at index [2]